# Install and import libraries

In [1]:
import folium
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon, MultiPolygon
from IPython.display import clear_output, display
import time

import os

In [4]:
# Mount Drive
#from google.colab import drive
#drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Set wd
os.chdir('/Users/benediktkorbach/Documents/GitHub/remote-sensing-of-parking-areas')

print("Working directory:", os.getcwd())

Working directory: /Users/benediktkorbach/Documents/GitHub/remote-sensing-of-parking-areas


# Create folium map display function

In [3]:
def create_folium_map(coord, service_stations = None, parking = None, zoom_start = 18, max_zoom = 23):
    """
    Create and display a folium map from coordinates based on German Web Map Services that includes service station and car/truck parking polygons

    Parameters:
    coord (list): [lat, lon] Coordinates.
    service_stations (geopandas df): Geopandas Dataframe of service stations containing service station area polygons.
    parking (geopandas df): Geopandas Dataframe containing truck and car parking spaces
    zoom_start (int): Zoom start level for Folium map.
    max_zoom (int): Maximum zooom level for Folium map.
    """

    # Instantiate folium map
    m = folium.Map(location=coord, zoom_start=zoom_start, max_zoom = max_zoom)

    def add_wms_layer(layer_name, wms_url, folium_base_map):
        """
        Add a WMS layer to a Folium map.

        Parameters:
        layer_name (str): The name assigned to layer.
        wms_url (str): the GetCapabilities url of the WMS layer.
        folium_base_map (folium.Map): The base map item.
        """

        # Define the WMS layer parameters
        wms_layer = folium.raster_layers.WmsTileLayer(
            url=wms_url,
            layers=layer_name,
            name=layer_name,
            fmt='image/png',
            transparent=True,
            overlay=True,
            control=False,
        )

        wms_layer.add_to(folium_base_map)

    # Define layers and WMS layer urls as dictionary
    wms_urls = {
        'WMS_NI_DOP': 'https://www.geobasisdaten.niedersachsen.de/doorman/noauth/wms_ni_dop?Request=GetCapabilities&Service=WMS', # Lower Saxony
        'bebb_dop20c': 'https://isk.geobasis-bb.de/mapproxy/dop20c/service/wms?', # Brandenburg & Berlin
        'th_dop': 'https://www.geoproxy.geoportal-th.de/geoproxy/services/DOP?version=1.1.1&', # Thuringia
        'lsa_lvermgeo_dop20_2': 'https://www.geodatenportal.sachsen-anhalt.de/wss/service/ST_LVermGeo_DOP_WMS_OpenData/guest?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetLegendGraphic&LAYER=lsa_lvermgeo_dop20_2&FORMAT=image/png', # Saxony-Anhalt
        'he_dop_rgb': 'https://gds-srv.hessen.de/cgi-bin/lika-services/ogc-free-images.ows?', # Hessia
        'sh_dop20_rgb': 'https://dienste.gdi-sh.de/WMS_SH_DOP20col_OpenGBD?' # Schleswig-Holstein
    }

    # Add all WMS layers to basemap
    for layer, url in wms_urls.items():
        add_wms_layer(layer, url, m)

    # Add service station are polygon to folium map in yellow
    for index, row in service_stations.iterrows():
        # Ensure that the geometry exists
        if row.geometry:
            # Convert the geometry to GeoJSON
            geo_json = folium.GeoJson(row.geometry.__geo_interface__,
                                      style_function=lambda feature: {
                                          "color": "yellow"
                                      },
            ).add_to(m)

    # Add truck and car parking polygons to folium map in green and blue
    for index, row in parking.iterrows():
        # Ensure that the geometry exists
        if row.geometry:
            # Convert the geometry to GeoJSON
            if row["type"] == "car":
                geo_json = folium.GeoJson(row.geometry.__geo_interface__,
                                          style_function=lambda feature: {
                                              "color": "green"
                                          },
                ).add_to(m)
            elif row["type"] == "truck":
                geo_json = folium.GeoJson(row.geometry.__geo_interface__,
                                          style_function=lambda feature: {
                                              "color": "blue"
                                          },
                ).add_to(m)
            else:
                raise ValueError("No parking type given.")

    # Create red dot on folium map to indicate the current service area
    lat = coord[0]
    lon = coord[1]
    folium.CircleMarker(location=[lat, lon],
                        radius=2,
                        weight=5,
                        color="red"
                        ).add_to(m)

    # Display the map
    display(m)

In [4]:
def create_coord(polygon):
    """
    Create a bounding box [left, bottom, right, top] from a polygon.

    Parameters:
    polygon: (shapely.geometry.Polygon): The input polygon

    Returns:
    A list containing the bounding box coordinates [left, bottom, right, top].
    """

    # Get the bounding box coordinates
    bottom, left, top, right = polygon.bounds

    # Select the middle point of the bounding box
    lon = (bottom + top) / 2
    lat = (left + right) / 2

    # Return the middle point coordinates as a list
    return [lat, lon]

# Show service stations and save correct service stations to list

## Load GeoJSONs as geopandas dataframes

In [5]:
# Load the GeoJSON files containing service station polygons and truck/car parking polygons together with service station IDs (created in 01a_OSM_rest_stops.ipynb)
all_parkings = gpd.read_file("02_data_acquisition/parking_data/all_parkings.geojson") # polygons of car and truck parking space
rest_stations = gpd.read_file("02_data_acquisition/parking_data/rest_stations.geojson") # polygons and ids of rest stations

## Inspect geopandas dataframes

In [6]:
# Inspect all_parkings
all_parkings.head()

,id_car,@id,name,id_rest,type,geometry
0,relation/13277622,relation/13277622,None,lon_10.2703215_lat_53.3301407,car,"MULTIPOLYGON (((10.26903 53.33039, 10.26926 53..."
1,relation/13277623,relation/13277623,None,lon_10.2712414_lat_53.331074,car,"MULTIPOLYGON (((10.26848 53.33228, 10.26872 53..."
2,relation/15482166,relation/15482166,None,lon_12.140788_lat_52.2431929,car,"MULTIPOLYGON (((12.14216 52.24271, 12.14213 52..."
3,relation/15482306,relation/15482306,None,lon_12.149456_lat_52.2449316,car,"MULTIPOLYGON (((12.14713 52.24497, 12.14717 52..."
4,way/31129920,way/31129920,Nord-Ostsee-Kanal,lon_9.323161_lat_54.0686721,car,"POLYGON ((9.32127 54.06898, 9.32314 54.06861, ..."


In [7]:
# Inspect rest_stations
rest_stations.head()

,id,@id,highway,name,geometry
0,lon_11.2442867_lat_51.3248863,relation/3947321,rest_area,Hohe Schrecke West,"MULTIPOLYGON (((11.24429 51.32489, 11.24430 51..."
1,lon_13.5458137_lat_52.7129474,relation/4672225,rest_area,Probstheide,"POLYGON ((13.54581 52.71295, 13.54556 52.71258..."
2,lon_13.549042_lat_52.7092396,relation/4672228,rest_area,Ladeburger Heide,"POLYGON ((13.54904 52.70924, 13.54912 52.70919..."
3,lon_8.6630029_lat_50.2542348,way/22568867,rest_area,Schäferborn,"POLYGON ((8.66300 50.25423, 8.66302 50.25237, ..."
4,lon_8.6624068_lat_50.2544468,way/22568868,rest_area,Spießwald,"POLYGON ((8.66241 50.25445, 8.66205 50.25369, ..."


In [8]:
# Inspect dimensions of the geopandas dataframes
print("all_parkings contains", all_parkings.shape[0], "unique car and truck parking spaces.")
print("rest_stations contains", rest_stations.shape[0], "unique rest stations.")

all_parkings contains 1606 unique car and truck parking spaces.
rest_stations contains 506 unique rest stations.


## Select correctly labelled service stations

In [45]:
# For every service station in rest_stations, create and show a folium map and input whether the service station and the car/truck parking areas are correctly labelled.

# Create empty lists and set index_counter to zero
index_counter = 0

service_stations_correct = []
service_stations_wrong = []

# Loop over all the service stations, shos corresponding folium map and input whether annotation is accurate
for index, row in rest_stations.iterrows():
    coord = create_coord(row[-1]) # create coordinates from service station polygon
    name = row["name"]
    id = row["id"]

    # Store folium map in a variable
    folium_map = create_folium_map(coord, service_stations=rest_stations, parking=all_parkings)

    # Give map time to render and not cover input box
    time.sleep(1)
    print("\n")

    # Clear the previous output and display the new map
    display(folium_map)
    clear_output(wait=True)

    # Ask user to input whether the service station and car/truck parking annotations are accurate
    user_input = input(f"Are the parking spaces at {name} (id: {id}) correctly labelled? Yes: \"y\". No: \"n\". Exit: \"exit\"")

    if user_input == "y":
        service_stations_correct.append(row["id"])
    elif user_input == "n":
        service_stations_wrong.append(row["id"])
    elif user_input == "exit":
        print(f"Next station to be verified: {index_counter}")
        break
    else:
        print("Invalid user_input. Please enter 'y', 'n', or 'exit'.")

    # Delete the map object to free up memory
    del folium_map

    index_counter += 1

    # Show user lastly checked service station
    print("Last image checked:", row["id"], index_counter - 1)

NameError: name 'create_coord' is not defined

In [ ]:
# Show list of correctly annotated service stations

service_stations_correct

['lon_8.6630029_lat_50.2542348',
 'lon_11.3531491_lat_50.9378362',
 'lon_9.6300119_lat_53.8303266',
 'lon_9.6297338_lat_53.8301666',
 'lon_11.0102562_lat_50.8353109',
 'lon_12.1106921_lat_51.4564157',
 'lon_12.1076965_lat_51.4562754',
 'lon_10.1688328_lat_53.3457924',
 'lon_12.0315302_lat_51.2103528',
 'lon_10.7298729_lat_50.5605634',
 'lon_8.7976979_lat_50.4392699',
 'lon_8.7960067_lat_50.4414072',
 'lon_9.0037966_lat_50.1997276',
 'lon_9.006114_lat_50.1972118',
 'lon_10.16857_lat_53.3461669',
 'lon_10.444298_lat_50.9932399',
 'lon_10.4367549_lat_50.9933128',
 'lon_10.6579729_lat_51.9212477',
 'lon_9.9309273_lat_54.1889457',
 'lon_10.0774066_lat_53.2322212',
 'lon_10.0214344_lat_54.2446698',
 'lon_8.4094366_lat_50.0433047',
 'lon_8.409903_lat_50.0435443',
 'lon_12.484777_lat_51.9700491',
 'lon_12.6318286_lat_52.0445002',
 'lon_9.6950885_lat_52.4313338',
 'lon_10.6158177_lat_53.5536922',
 'lon_10.6284325_lat_53.5509146',
 'lon_9.43935_lat_54.6368415',
 'lon_13.8513592_lat_51.4570059',


In [ ]:
# Show list of wrongly annotated service stations

service_stations_wrong

['lon_11.2442867_lat_51.3248863',
 'lon_13.5458137_lat_52.7129474',
 'lon_13.549042_lat_52.7092396',
 'lon_8.6624068_lat_50.2544468',
 'lon_8.6184146_lat_49.7896285',
 'lon_9.7581375_lat_52.4268193',
 'lon_10.2712414_lat_53.331074',
 'lon_8.633307_lat_50.0540941',
 'lon_9.2506723_lat_54.1383328',
 'lon_10.2703215_lat_53.3301407',
 'lon_8.9264579_lat_50.3435768',
 'lon_8.9282225_lat_50.3458508',
 'lon_9.930895_lat_52.2392839',
 'lon_9.9290741_lat_52.2397704',
 'lon_9.9118833_lat_54.3141951',
 'lon_9.9181171_lat_54.315654',
 'lon_14.1236975_lat_51.7547827',
 'lon_10.1741282_lat_51.4096944',
 'lon_10.7778756_lat_51.4723257',
 'lon_11.7243104_lat_51.3999369',
 'lon_12.2147233_lat_51.2415014',
 'lon_9.6690837_lat_51.3960842',
 'lon_9.5648832_lat_51.3206232',
 'lon_9.7765986_lat_51.39495',
 'lon_9.7800847_lat_51.3955138',
 'lon_9.4159049_lat_51.2726438',
 'lon_9.4159061_lat_51.2721518',
 'lon_9.3682659_lat_51.2848607',
 'lon_11.0524687_lat_52.2204778',
 'lon_9.2941255_lat_51.3122625',
 'lon_

In [ ]:
# Print number of correctly and wrongly labelled service stations

print("Number of correctly labelled service stations:", len(service_stations_correct))
print("Number of wrongly labelled service stations:", len(service_stations_wrong))

Number of correctly labelled service stations: 267
Number of wrongly labelled service stations: 239


## Save lists as CSVs

In [ ]:
def save_list_to_csv(list_input, name, file_path):
    """
    Saves list to CSV.

    Parameters:
    list_input (list): The list input.
    name (str): The name assigned to the list.
    file_path (str): The path where the csv is to be stored.

    Returns:
    string: Saving confirmation.
    """

    dictionary = {"id_rest": list_input}
    df = pd.DataFrame(dictionary)
    df.to_csv(f"{file_path}/{name}.csv", index=False)

    return f"{name} saved in {file_path}/{name}.csv."

In [ ]:
# Save lists as CSVs

save_list_to_csv(service_stations_correct, "verified_service_stations", "/content/drive/MyDrive/Master Thesis/01 Data Acquisition/")
save_list_to_csv(service_stations_wrong, "wrongly_labelled_service_stations", "/content/drive/MyDrive/Master Thesis/01 Data Acquisition/")

'wrongly_labelled_service_stations saved in /content/drive/MyDrive/Master Thesis/01 Data Acquisition//wrongly_labelled_service_stations.csv.'

# Change polygon labels

In [9]:
def create_folium_map(service_station, parking, zoom_start=18, max_zoom=23):
    """
    Create and display a folium map for a single service station showing the service station 
    and the parking polygons within its area, with numbered parking spaces on the map and details printed below.

    Parameters:
    service_station (Series): A Pandas Series representing a single service station with a geometry attribute.
    parking (geopandas df): Geopandas DataFrame containing truck and car parking spaces.
    zoom_start (int): Zoom start level for Folium map.
    max_zoom (int): Maximum zoom level for Folium map.
    """
    # Initialize map at the service station's centroid
    m = folium.Map(location=[service_station.geometry.centroid.y, service_station.geometry.centroid.x], zoom_start=zoom_start, max_zoom=max_zoom)
    
    def add_wms_layer(layer_name, wms_url, folium_base_map):
        """
        Add a WMS layer to a Folium map.

        Parameters:
        layer_name (str): The name assigned to layer.
        wms_url (str): the GetCapabilities url of the WMS layer.
        folium_base_map (folium.Map): The base map item.
        """

        # Define the WMS layer parameters
        wms_layer = folium.raster_layers.WmsTileLayer(
            url=wms_url,
            layers=layer_name,
            name=layer_name,
            fmt="image/png",
            transparent=True,
            overlay=True,
            control=False,
        )

        wms_layer.add_to(folium_base_map)

    # Define layers and WMS layer urls as dictionary
    wms_urls = {
        'WMS_NI_DOP': 'https://www.geobasisdaten.niedersachsen.de/doorman/noauth/wms_ni_dop?Request=GetCapabilities&Service=WMS', # Lower Saxony
        'bebb_dop20c': 'https://isk.geobasis-bb.de/mapproxy/dop20c/service/wms?', # Brandenburg & Berlin
        'th_dop': 'https://www.geoproxy.geoportal-th.de/geoproxy/services/DOP?version=1.1.1&', # Thuringia
        'lsa_lvermgeo_dop20_2': 'https://www.geodatenportal.sachsen-anhalt.de/wss/service/ST_LVermGeo_DOP_WMS_OpenData/guest?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetLegendGraphic&LAYER=lsa_lvermgeo_dop20_2&FORMAT=image/png', # Saxony-Anhalt
        'he_dop_rgb': 'https://gds-srv.hessen.de/cgi-bin/lika-services/ogc-free-images.ows?', # Hessia
        'sh_dop20_rgb': 'https://dienste.gdi-sh.de/WMS_SH_DOP20col_OpenGBD?' # Schleswig-Holstein
    }

    # Add all WMS layers to basemap
    for layer, url in wms_urls.items():
        add_wms_layer(layer, url, m)

    # Filter parking spaces within the service station polygon
    within_service_area = parking[parking.id_rest == service_station.id]
    
    parking_space_number = 1  # Initialize parking space identifier
    parking_space_details = []  # Initialize list to store parking details for printing

    # Add service station area polygon to the map in yellow
    folium.GeoJson(
        service_station.geometry.__geo_interface__,
        style_function=lambda feature: {"color": "yellow"},
        tooltip=f"Service Station"
    ).add_to(m)

    # Add filtered parking polygons to the map with numbers
    for _, parking_row in within_service_area.iterrows():
        color = "red" if parking_row["type"] == "car" else "blue" if parking_row["type"] == "truck" else "green" if parking_row["type"] == "quick" else "black"
        parking_label = f"{parking_space_number}. {parking_row['type']} parking"
        
        if parking_row["type"] == "car":
            folium.GeoJson(
                parking_row.geometry.__geo_interface__,
                style_function=lambda feature: {"color": "red"},
                tooltip=parking_label
                ).add_to(m)
        elif parking_row["type"] == "truck":
            folium.GeoJson(
                parking_row.geometry.__geo_interface__,
                style_function=lambda feature: {"color": "blue"},
                tooltip=parking_label
                ).add_to(m)
        elif parking_row["type"] == "quick":
            folium.GeoJson(
                parking_row.geometry.__geo_interface__,
                style_function=lambda feature: {"color": "green"},
                tooltip=parking_label
                ).add_to(m)
        else:
            raise ValueError("No parking type given.")
        
        # Store parking space details for printing
        parking_space_details.append(f"{parking_space_number}: Type - {parking_row['type']}, ID - {parking_row['id_rest']}")
        
        parking_space_number += 1  # Increment the parking space counter

    # Display the map for the current service station
    display(m)

    # Print parking space details below the map
    print("\nParking Space Details:")
    for detail in parking_space_details:
        print(detail)

    # Give time for the map to be rendered
    #time.sleep(1)  # Adjust sleep time as needed

    # Optionally clear the output to keep the notebook tidy
    clear_output(wait=True)

In [10]:
# Test create_folium_map
test_service_station = rest_stations.iloc[3]
test_parking = all_parkings

create_folium_map(test_service_station, test_parking)


Parking Space Details:
1: Type - car, ID - lon_8.6630029_lat_50.2542348
2: Type - truck, ID - lon_8.6630029_lat_50.2542348
3: Type - truck, ID - lon_8.6630029_lat_50.2542348


In [49]:
# Initialize verification state
index_counter = 0
service_stations_correct = []
service_stations_wrong = []

def verify_and_label_service_stations(service_stations, parking):
    global index_counter  # To keep track of the current index across the function calls

    for index, row in service_stations.iterrows():
        # Skip already verified stations
        if index < index_counter:
            continue
        
        label_updated = True

        while label_updated == True:
            # Display the map for the current service station
            create_folium_map(row, parking)

            # Handling label change request
            change_label = input(f"Do you want to change the label of any parking space at {row['name']} (id: {row['id']})? (y/n)").strip().lower()
            if change_label == "y":
                parking_marker_number = input("Enter the parking spot number as shown on the map: ").strip()
                new_type_input = input("Enter the new parking type (c/t/q): ").strip().lower()
                if new_type_input in ["c", "t", "q"]:
                    new_type = "car" if new_type_input == "c" else "truck" if new_type_input == "t" else "quick" if new_type_input == "q" else "unknown"  # Convert user input to parking type

                    # Assuming parking spaces are in the same order as displayed
                    try:
                        parking_space_index = parking[parking["id_rest"] == row["id"]].index[int(parking_marker_number) - 1]
                        parking.at[parking_space_index, "type"] = new_type
                        clear_output(wait=True)
                    except (IndexError, ValueError):
                        print("Invalid parking spot number. Please enter a valid number as shown on the map.")
                else:
                    print("Invalid parking type. Please enter 'c' for car, 't' for truck, or 'q' for quick parking.")
            if change_label == "n":
                label_updated = False
            else:
                print("Invalid input. Please enter 'y' or 'n'.")
        
        # Asking for verification of parking labels after potentially updating them
        user_input = input(f"Are the parking spaces at {row['name']} (id: {row['id']}) correctly labelled now? (y/n/exit)").strip().lower()

        # Update lists based on user input
        if user_input == "y":
            service_stations_correct.append(row["id"])
        elif user_input == "n":
            service_stations_wrong.append(row["id"])
        elif user_input == "exit":
            print(f"Exiting. Next station to be verified: {index_counter}, {service_stations.iloc[index_counter]['name']}, {service_stations.iloc[index_counter + 1]['id']}")
            index_counter = index_counter - 1 # Adjust the index counter to re-verify the current station
            break  # Exit the loop
        else:
            print("Invalid input. Please enter 'y', 'n', or 'exit'.")
                
        # Update the index counter to move to the next station
        index_counter = index + 1

        # Clear output to prepare for the next service station verification
        clear_output(wait=True)

        # Show which service station was last checked
        print("Last service station checked:", row["name"], row["id"], "Index:", index_counter - 1)
    
    return service_stations_correct, service_stations_wrong, parking

In [50]:
# Call the verification function with your dataframes
service_stations_correct, service_stations_wrong, parking_corrected = verify_and_label_service_stations(rest_stations, all_parkings)

Last service station checked: Rastplatz Bissendorf Süd lon_9.7695159_lat_52.5102873 Index: 505


In [51]:
service_stations_wrong

['lon_11.2442867_lat_51.3248863',
 'lon_13.5458137_lat_52.7129474',
 'lon_13.549042_lat_52.7092396',
 'lon_8.6624068_lat_50.2544468',
 'lon_8.6184146_lat_49.7896285',
 'lon_9.7581375_lat_52.4268193',
 'lon_8.633307_lat_50.0540941',
 'lon_10.2703215_lat_53.3301407',
 'lon_9.930895_lat_52.2392839',
 'lon_9.9290741_lat_52.2397704',
 'lon_10.6579729_lat_51.9212477',
 'lon_14.1236975_lat_51.7547827',
 'lon_10.1741282_lat_51.4096944',
 'lon_10.6238862_lat_51.4340877',
 'lon_10.7778756_lat_51.4723257',
 'lon_11.7243104_lat_51.3999369',
 'lon_12.2147233_lat_51.2415014',
 'lon_9.6690837_lat_51.3960842',
 'lon_9.5648832_lat_51.3206232',
 'lon_9.7765986_lat_51.39495',
 'lon_9.4159049_lat_51.2726438',
 'lon_9.3682659_lat_51.2848607',
 'lon_9.3696302_lat_51.2844243',
 'lon_9.2941255_lat_51.3122625',
 'lon_9.1600031_lat_51.4173784',
 'lon_9.1613741_lat_51.4145048',
 'lon_7.2187753_lat_52.2848132',
 'lon_9.0331918_lat_51.4840659',
 'lon_14.6664657_lat_51.6935159',
 'lon_9.4270122_lat_53.2544235',
 'l

In [52]:
service_stations_correct

['lon_8.6630029_lat_50.2542348',
 'lon_11.3531491_lat_50.9378362',
 'lon_10.2712414_lat_53.331074',
 'lon_9.6300119_lat_53.8303266',
 'lon_9.6297338_lat_53.8301666',
 'lon_9.2506723_lat_54.1383328',
 'lon_11.0102562_lat_50.8353109',
 'lon_12.1106921_lat_51.4564157',
 'lon_12.1076965_lat_51.4562754',
 'lon_10.1688328_lat_53.3457924',
 'lon_12.0315302_lat_51.2103528',
 'lon_10.7298729_lat_50.5605634',
 'lon_8.7976979_lat_50.4392699',
 'lon_8.7960067_lat_50.4414072',
 'lon_9.0037966_lat_50.1997276',
 'lon_9.006114_lat_50.1972118',
 'lon_8.9264579_lat_50.3435768',
 'lon_8.9282225_lat_50.3458508',
 'lon_10.16857_lat_53.3461669',
 'lon_10.444298_lat_50.9932399',
 'lon_10.4367549_lat_50.9933128',
 'lon_9.9309273_lat_54.1889457',
 'lon_10.0774066_lat_53.2322212',
 'lon_10.0214344_lat_54.2446698',
 'lon_8.4094366_lat_50.0433047',
 'lon_8.409903_lat_50.0435443',
 'lon_12.484777_lat_51.9700491',
 'lon_12.6318286_lat_52.0445002',
 'lon_9.6950885_lat_52.4313338',
 'lon_10.6158177_lat_53.5536922',
 

In [53]:
parking_corrected

,id_car,@id,name,id_rest,type,geometry
0,relation/13277622,relation/13277622,None,lon_10.2703215_lat_53.3301407,car,"MULTIPOLYGON (((10.26903 53.33039, 10.26926 53..."
1,relation/13277623,relation/13277623,None,lon_10.2712414_lat_53.331074,car,"MULTIPOLYGON (((10.26848 53.33228, 10.26872 53..."
2,relation/15482166,relation/15482166,None,lon_12.140788_lat_52.2431929,truck,"MULTIPOLYGON (((12.14216 52.24271, 12.14213 52..."
3,relation/15482306,relation/15482306,None,lon_12.149456_lat_52.2449316,car,"MULTIPOLYGON (((12.14713 52.24497, 12.14717 52..."
4,way/31129920,way/31129920,Nord-Ostsee-Kanal,lon_9.323161_lat_54.0686721,car,"POLYGON ((9.32127 54.06898, 9.32314 54.06861, ..."
...,...,...,...,...,...,...
1601,None,way/1194289084,None,lon_11.0467665_lat_51.4688222,truck,"POLYGON ((11.04380 51.46788, 11.04414 51.46784..."
1602,None,way/1194289085,None,lon_11.0467665_lat_51.4688222,truck,"POLYGON ((11.04439 51.46815, 11.04445 51.46815..."
1603,None,way/1194289086,None,lon_11.0467665_lat_51.4688222,truck,"POLYGON ((11.04503 51.46846, 11.04506 51.46846..."
1604,None,way/1194289087,None,lon_11.0467665_lat_51.4688222,quick,"POLYGON ((11.04375 51.46758, 11.04399 51.46763..."


In [54]:
# Print number of correctly and wrongly labelled service stations

print("Number of correctly labelled service stations:", len(service_stations_correct))
print("Number of wrongly labelled service stations:", len(service_stations_wrong))

Number of correctly labelled service stations: 296
Number of wrongly labelled service stations: 208


In [55]:
def save_list_to_csv(list_input, name, file_path):
    """
    Saves list to CSV.

    Parameters:
    list_input (list): The list input.
    name (str): The name assigned to the list.
    file_path (str): The path where the csv is to be stored.

    Returns:
    string: Saving confirmation.
    """

    dictionary = {"id_rest": list_input}
    df = pd.DataFrame(dictionary)
    df.to_csv(f"{file_path}/{name}.csv", index=False)

    return f"{name} saved in {file_path}/{name}.csv."

In [56]:
# Save lists as CSVs

save_list_to_csv(service_stations_correct, "verified_service_stations_incl_quick", "02_data_acquisition/verified_parking_data/")
save_list_to_csv(service_stations_wrong, "wrongly_labelled_service_stations_incl_quick", "02_data_acquisition/verified_parking_data/")



'wrongly_labelled_service_stations_incl_quick saved in 02_data_acquisition/verified_parking_data//wrongly_labelled_service_stations_incl_quick.csv.'

In [38]:
# Save parking_corrected as GeoJSON
parking_corrected.to_file("02_data_acquisition/verified_parking_data/parking_corrected_incl_quick.geojson", driver="GeoJSON")